# Keras - Lab

## Introduction

In this lab you'll once again build a neural network, but this time you will be using Keras to do a lot of the heavy lifting.


## Objectives

You will be able to:

- Build a neural network using Keras
- Evaluate performance of a neural network using Keras

## Required Packages

We'll start by importing all of the required packages and classes.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn import preprocessing
from keras.preprocessing.text import Tokenizer
from keras import models
from keras import layers
from keras import optimizers

## Load the data

In this lab you will be classifying bank complaints available in the `'Bank_complaints.csv'` file.

In [ ]:
# Import data
df = None

# Inspect data
print(df.info())
df.head()

As mentioned earlier, your task is to categorize banking complaints into various predefined categories. Preview what these categories are and what percent of the complaints each accounts for.

In [ ]:
# Your code here

## Preprocessing

Before we build our neural network, we need to do several preprocessing steps. First, we will create word vector counts (a bag of words type representation) of our complaints text. Next, we will change the category labels to integers. Finally, we will perform our usual train-test split before building and training our neural network using Keras. With that, let's start munging our data!

## One-hot encoding of the complaints

Our first step again is to transform our textual data into a numerical representation. As we saw in some of our previous lessons on NLP, there are many ways to do this. Here, we'll use the `Tokenizer()` class from the `preprocessing.text` sub-module of the Keras package.   

As with our previous work using NLTK, this will transform our text complaints into word vectors. (Note that the method of creating a vector is different from our previous work with NLTK; as you'll see, word order will be preserved as opposed to a bag of words representation). In the below code, we'll only keep the 2,000 most common words and use one-hot encoding.



In [ ]:
# As a quick preliminary, briefly review the docstring for keras.preprocessing.text.Tokenizer
Tokenizer?

In [ ]:
# ⏰ This cell may take about thirty seconds to run

# Raw text complaints
complaints = df['Consumer complaint narrative']

# Initialize a tokenizer
tokenizer = Tokenizer(num_words=2000)

# Fit it to the complaints
tokenizer.fit_on_texts(complaints)

# Generate sequences
sequences = tokenizer.texts_to_sequences(complaints)
print('sequences type:', type(sequences))

# Similar to sequences, but returns a numpy array
one_hot_results= tokenizer.texts_to_matrix(complaints, mode='binary')
print('one_hot_results type:', type(one_hot_results))

# Useful if we wish to decode (more explanation below)
word_index = tokenizer.word_index

# Tokens are the number of unique words across the corpus
print('Found %s unique tokens.' % len(word_index))

# Our coded data
print('Dimensions of our coded results:', np.shape(one_hot_results))

## Decoding Word Vectors

As a note, you can also decode these vectorized representations of the reviews. The `word_index` variable, defined above, stores the mapping from the label number to the actual word. Somewhat tediously, we can turn this dictionary inside out and map it back to our word vectors, giving us roughly the original complaint back. (As you'll see, the text won't be identical as we limited ourselves to top 2000 words.)

## Python Review / Mini Challenge

While a bit tangential to our main topic of interest, we need to reverse our current dictionary `word_index` which maps words from our corpus to integers. In decoding our `one_hot_results`, we will need to create a dictionary of these integers to the original words. Below, take the `word_index` dictionary object and change the orientation so that the values are keys and the keys values. In other words, you are transforming something of the form {A:1, B:2, C:3} to {1:A, 2:B, 3:C}.

In [ ]:
# Your code here
reverse_index = None

## Back to Decoding Our Word Vectors...

In [ ]:
comment_idx_to_preview = 19
print('Original complaint text:')
print(complaints[comment_idx_to_preview])
print('\n\n')

# The reverse_index cell block above must be complete in order for this cell block to successively execute
decoded_review = ' '.join([reverse_index.get(i) for i in sequences[comment_idx_to_preview]])
print('Decoded review from Tokenizer:')
print(decoded_review)

## Convert the Products to Numerical Categories

On to step two of our preprocessing: converting our descriptive categories into integers.

In [ ]:
product = df['Product']

# Initialize
le = preprocessing.LabelEncoder()
le.fit(product)
print('Original class labels:')
print(list(le.classes_))
print('\n')
product_cat = le.transform(product)

# If you wish to retrieve the original descriptive labels post production
# list(le.inverse_transform([0, 1, 3, 3, 0, 6, 4]))

print('New product labels:')
print(product_cat)
print('\n')

# Each row will be all zeros except for the category for that observation
print('One hot labels; 7 binary columns, one for each of the categories.')
product_onehot = to_categorical(product_cat)
print(product_onehot)
print('\n')

print('One hot labels shape:')
print(np.shape(product_onehot))

## Train-test split

Now for our final preprocessing step: the usual train-test split.

In [ ]:
random.seed(123)
test_index = random.sample(range(1,10000), 1500)

test = one_hot_results[test_index]
train = np.delete(one_hot_results, test_index, 0)

label_test = product_onehot[test_index]
label_train = np.delete(product_onehot, test_index, 0)

print('Test label shape:', np.shape(label_test))
print('Train label shape:', np.shape(label_train))
print('Test shape:', np.shape(test))
print('Train shape:', np.shape(train))

## Building the network

Let's build a fully connected (Dense) layer network with relu activation in Keras. You can do this using: `Dense(16, activation='relu')`.

In this example, use two hidden layers with 50 units in the first layer and 25 in the second, both with a `'relu'` activation function. Because we are dealing with a multiclass problem (classifying the complaints into 7 categories), we use a use a `'softmax'` classifier in order to output 7 class probabilities per case.  

In [ ]:
# Initialize a sequential model
model = None

# Two layers with relu activation



# One layer with softmax activation


## Compiling the model

Now, compile the model! This time, use `'categorical_crossentropy'` as the loss function and stochastic gradient descent, `'SGD'` as the optimizer. As in the previous lesson, include the accuracy as a metric.

In [ ]:
# Compile the model


## Training the model

In the compiler, you'll be passing the optimizer (SGD = stochastic gradient descent), loss function, and metrics. Train the model for 120 epochs in mini-batches of 256 samples.

_Note:_ ⏰ _Your code may take about one to two minutes to run._

In [ ]:
# Train the model
history = None

Recall that the dictionary `history` has two entries: the loss and the accuracy achieved using the training set.

In [ ]:
history_dict = history.history
history_dict.keys()

## Plot the results

As you might expect, we'll use our `matplotlib` for graphing. Use the data stored in the `history_dict` above to plot the loss vs epochs and the accuracy vs epochs.

In [ ]:
# Plot the loss vs the number of epoch

In [ ]:
# Plot the training accuracy vs the number of epochs


It seems like we could just keep on going and accuracy would go up!

## Make predictions

Finally, it's time to make predictions. Use the relevant method discussed in the previous lesson to output (probability) predictions for the test set.

In [ ]:
# Output (probability) predictions for the test set
y_hat_test = None

## Evaluate Performance

Finally, print the loss and accuracy for both the train and test sets of the final trained model.

In [ ]:
# Print the loss and accuracy for the training set
results_train = None
results_train

In [ ]:
# Print the loss and accuracy for the test set
results_test = None
results_test

We can see that the training set results are really good, and the test set results seem to be even better. In general, this type of result will be rare, as train set results are usually at least a bit better than test set results.

## Summary

Congratulations! In this lab, you built a neural network thanks to the tools provided by Keras! In upcoming lessons and labs we'll continue to investigate further ideas regarding how to tune and refine these models for increased accuracy and performance.

In [1]:
#Me trying some stuff out from a short video on building NN's
import numpy as np
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse

In [2]:
X, y = make_regression(n_features = 2,
                       n_informative = 1,
                       noise = 100,
                       random_state = 42)

In [3]:
linreg = LinearRegression()
linreg.fit(X, y).score(X, y)

0.4144461358278726

In [5]:
X.shape

(100, 2)

In [7]:
y.shape

(100,)

In [8]:
y[0:10]

array([ -86.45119104,  110.51001716,  -57.28377464, -256.31443259,
         60.82745498,  -58.76914197,    7.00903069,  114.58447502,
        310.19242342,   16.59536815])

In [9]:
X_2, y_2 = make_regression(n_features = 2,
                       n_informative = 1,
                       noise = 50,
                       random_state = 42)
linreg.fit(X_2, y_2).score(X_2, y_2)

0.7131845881420126

In [11]:
np.random.rand(2)

array([0.24844698, 0.99362556])

In [12]:
np.random.rand(4,3)

array([[0.73297071, 0.56669941, 0.09732822],
       [0.58525376, 0.04847888, 0.07773121],
       [0.0337703 , 0.28439405, 0.71779533],
       [0.04523771, 0.4288909 , 0.11838   ]])

In [13]:
X

array([[-1.41537074, -0.42064532],
       [ 0.52194157,  0.29698467],
       [-0.88951443, -0.81581028],
       [-0.88385744,  0.15372511],
       [ 0.73846658,  0.17136828],
       [-0.26465683,  2.72016917],
       [ 1.14282281,  0.75193303],
       [ 0.36139561,  1.53803657],
       [ 0.81252582,  1.35624003],
       [-0.22346279,  0.71400049],
       [-1.10633497, -1.19620662],
       [-0.84679372, -1.51484722],
       [-0.60063869, -0.29169375],
       [ 0.00511346, -0.23458713],
       [-0.23415337, -0.23413696],
       [-1.60748323,  0.18463386],
       [ 0.09707755,  0.96864499],
       [ 0.32408397, -0.38508228],
       [-0.39210815, -1.46351495],
       [-0.07201012,  1.0035329 ],
       [-0.46341769, -0.46572975],
       [ 1.8861859 ,  0.17457781],
       [ 1.57921282,  0.76743473],
       [ 0.09965137, -0.50347565],
       [ 0.81351722, -1.23086432],
       [-1.1913035 ,  0.65655361],
       [ 0.58685709,  2.19045563],
       [-1.32818605,  0.19686124],
       [-0.11564828,

In [17]:
a = np.random.rand(1,3)
b = np.random.rand(1,3)
a, b

(array([[0.70586544, 0.6539683 , 0.87164084]]),
 array([[0.55320201, 0.72374856, 0.67350014]]))

In [20]:
np.hstack((a, b))

array([[0.70586544, 0.6539683 , 0.87164084, 0.55320201, 0.72374856,
        0.67350014]])

In [21]:
np.vstack((a,b))

array([[0.70586544, 0.6539683 , 0.87164084],
       [0.55320201, 0.72374856, 0.67350014]])

In [24]:
c = np.random.rand(3,1)
d = np.random.rand(3,1)
c, d

(array([[0.45070857],
        [0.44165713],
        [0.96236974]]),
 array([[0.42444334],
        [0.36153809],
        [0.97200155]]))

In [25]:
np.hstack((c,d))

array([[0.45070857, 0.42444334],
       [0.44165713, 0.36153809],
       [0.96236974, 0.97200155]])

In [26]:
np.vstack((c,d))

array([[0.45070857],
       [0.44165713],
       [0.96236974],
       [0.42444334],
       [0.36153809],
       [0.97200155]])